In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import lxml

req = requests.get("https://www.espncricinfo.com/records/season/team-match-results/2024-2024?trophy=89")
soup = bs(req.content)

In [9]:
summary_table = soup.find('table')
summary_df = pd.read_html(str(summary_table))[0]

<ipython-input-9-77dd619802d6>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  summary_df = pd.read_html(str(summary_table))[0]


# Getting Individual Matches

In [ ]:
# Creating an array containing the links of scorecards of all matches
match_links = summary_table.find_all('a',)

href_list = []
for link in match_links:
  href = link.get('href')
  href_list.append(href)

links = href_list[1::2]

# First Match Scorecard: